In [29]:
import os
import sys
import operator
import math
import csv

posInDir = "data/pos"
negOutDir = "data/neg"

if (not os.path.isdir(posInDir)):
    print(f"Error: path \"{posInDir}\" is not a valid directory!")
    exit(1)

if (not os.path.exists(negOutDir)):
    os.makedirs(negOutDir)

In [42]:
# Count frequency of sequence lengths to determine ideal cutoff length

posLenDict = {}
negLenDict = {}

for file in os.listdir(posInDir):
  filename = os.fsdecode(file)
  filepath = os.path.join(posInDir, filename)

  if (filename.endswith(".bed") and not "_trimmed" in filename):
    with open(filepath, "r") as infile:
      reader = csv.reader(infile, delimiter="\t")

      prevID = ""
      prevStart = ""
      prevEnd = ""
      
      for line in reader:
        curID = line[0]
        curStart = line[1]
        curEnd = line[2]

        posLen = int(curEnd) - int(curStart) + 1
        if posLen in posLenDict:
          posLenDict[posLen] += 1
        else:
          posLenDict[posLen] = 1

        if (prevID != "" and prevStart != "" and prevEnd != "" and curID == prevID and curStart != prevEnd and curStart != curEnd):
          newID = curID
          newStart = int(prevEnd) + 1
          newEnd = int(curStart) - 1
          if (newEnd > newStart):
            negLen = newEnd - newStart + 1
            if negLen in negLenDict:
              negLenDict[negLen] += 1
            else:
              negLenDict[negLen] = 1
        prevID = curID
        prevStart = curStart
        prevEnd = curEnd

posLenList = sorted(posLenDict.items(), key=operator.itemgetter(1), reverse=True)
negLenList = sorted(negLenDict.items(), key=operator.itemgetter(1), reverse=True)

posCountTotal = sum(i[1] for i in posLenList)
negCountTotal = sum(i[1] for i in negLenList)
combinedCountTotal = posCountTotal + negCountTotal

combinedCountTarget = math.floor(combinedCountTotal / 2)

posIdx = 0
negIdx = 0

combinedLenList = []
while (posIdx < len(posLenList) and negIdx < len(negLenList)):
  if (posLenList[posIdx][0] > negLenList[negIdx][0]):
    combinedLenList.append(posLenList[posIdx])
    posIdx += 1
  elif (posLenList[posIdx][0] == negLenList[negIdx][0]):
    combinedLenList.append([posLenList[posIdx][0], posLenList[posIdx][1] + negLenList[negIdx][1]])
    posIdx += 1
    negIdx += 1
  else:
    combinedLenList.append(negLenList[negIdx])
    negIdx += 1
while (posIdx < len(posLenList)):
  combinedLenList.append(posLenList[posIdx])
  posIdx += 1
while (negIdx < len(negLenList)):
  combinedLenList.append(negLenList[negIdx])
  negIdx += 1


remaining = combinedCountTotal
i = 0
while (remaining >= combinedCountTarget):
  remaining -= combinedLenList[i][1]
  if (remaining < combinedCountTarget):
    remaining += combinedLenList[i][1]
    break
  i += 1

cutoffLen = combinedLenList[i][0]

maxPosLen = sorted(posLenList, key=operator.itemgetter(0), reverse=True)[0][0]
maxNegLen = sorted(negLenList, key=operator.itemgetter(0), reverse=True)[0][0]

print(f"Max positive length: {maxPosLen}")
print(f"Max negative length: {maxNegLen}")
print(f"Cutoff length: {cutoffLen}")

Max positive length: 3744
Max negative length: 245889217
Cutoff length: 1066


In [43]:
# Trim positive files

linesWrittenList = []

for file in os.listdir(posInDir):
  filename = os.fsdecode(file)
  filepath = os.path.join(posInDir, filename)
  filename_noext = os.path.splitext(filename)[0]

  outfpath = filename_noext + "_trimmed.bed"
  outfpath = os.path.join("data/pos", outfpath)
  
  if (filename.endswith(".bed") and not "_trimmed" in filename):
    with open(filepath, "r") as infile, open(outfpath, "w") as outfile:
      reader = csv.reader(infile, delimiter="\t")
      writer = csv.writer(outfile, delimiter="\t")

      prevID = ""
      prevStart = ""
      prevEnd = ""
      
      i = 0
      j = 0
      for line in reader:
        curID = line[0]
        curStart = line[1]
        curEnd = line[2]

        posLen = int(curEnd) - int(curStart) + 1
        if (posLen <= cutoffLen):
          j += 1
          writer.writerow(line[:3])
        i += 1
      linesWrittenList.append([os.path.basename(outfpath), j, i])

linesWrittenList.sort(key=operator.itemgetter(0))

In [44]:
# Remove original untrimmed positive files

for file in os.listdir(posInDir):
  filename = os.fsdecode(file)
  filepath = os.path.join(posInDir, filename)
  if (filename.endswith(".bed") and not "_trimmed" in filename):
    os.remove(filepath)

# Rename trimmed files
for file in os.listdir(posInDir):
  filename = os.fsdecode(file)
  filepath = os.path.join(posInDir, filename)

  outname = filename[:filename.find("_trimmed")] + ".bed"
  outpath = os.path.join(posInDir, outname)
  if (filename.endswith("_trimmed.bed")):
    os.rename(filepath, outpath) 

In [45]:
#  Create Negative Bedfiles

for file in os.listdir(posInDir):
    filename = os.fsdecode(file)
    filename_noext = os.path.splitext(filename)[0]

    filepath = os.path.join(posInDir, filename)
    if (filename.endswith(".bed")):
        outpath = os.path.join(negOutDir, filename)
        print("Creating file \"", outpath, "\"...", sep="")
        
        prevID = ""
        prevStart = ""
        prevEnd = ""

        with open(filepath, "r") as infile, open(outpath, "w") as outfile:
            reader = csv.reader(infile, delimiter="\t")
            writer = csv.writer(outfile, delimiter="\t")
        
            for line in reader:
                curID = line[0]
                curStart = line[1]
                curEnd = line[2]

                if (prevID != "" and prevStart != "" and prevEnd != "" and curID == prevID and curStart != prevEnd and curStart != curEnd):
                    newID = curID
                    newStart = int(prevEnd) + 1
                    newEnd = int(curStart) - 1
                    seqLen = newEnd - newStart + 1
                    if (newEnd > newStart and seqLen >= cutoffLen):
                        seqLen = min(seqLen, cutoffLen)
                        newEnd = min(newEnd, newStart + seqLen - 1)
                        writer.writerow([str(newID), str(newStart), str(newEnd)])

                prevID = curID
                prevStart = curStart
                prevEnd = curEnd
                continue
        continue
    else:
        print("File \"", filepath, "\" is not a valid positive bedfile", sep='')
        continue

Creating file "data/neg/sample11.bed"...
Creating file "data/neg/sample39.bed"...
Creating file "data/neg/sample38.bed"...
Creating file "data/neg/sample10.bed"...
Creating file "data/neg/sample12.bed"...
Creating file "data/neg/sample13.bed"...
Creating file "data/neg/sample17.bed"...
Creating file "data/neg/sample16.bed"...
Creating file "data/neg/sample28.bed"...
Creating file "data/neg/sample14.bed"...
Creating file "data/neg/sample8.bed"...
Creating file "data/neg/sample9.bed"...
Creating file "data/neg/sample15.bed"...
Creating file "data/neg/sample29.bed"...
Creating file "data/neg/sample41.bed"...
Creating file "data/neg/sample40.bed"...
Creating file "data/neg/sample42.bed"...
Creating file "data/neg/sample43.bed"...
Creating file "data/neg/sample30.bed"...
Creating file "data/neg/sample24.bed"...
Creating file "data/neg/sample18.bed"...
Creating file "data/neg/sample4.bed"...
Creating file "data/neg/sample5.bed"...
Creating file "data/neg/sample19.bed"...
Creating file "data/